# Imports

In [73]:
import numpy as np
from math import pi
import pandas as pd
from scipy.spatial import distance
from scipy.linalg import norm
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
import sqlite3

from qiskit.quantum_info import Kraus, SuperOp
from qiskit.providers.aer import (
    AerSimulator,
    QasmSimulator
)
from qiskit.providers.ibmq.job import job_monitor
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, QuantumRegister, ClassicalRegister, execute
from qiskit.quantum_info.analysis.distance import hellinger_distance

# Import from Qiskit Aer noise module
from qiskit.providers.aer.noise import (
    NoiseModel, 
    QuantumError, 
    ReadoutError, 
    pauli_error, 
    depolarizing_error, 
    thermal_relaxation_error
)

# Loading your IBM Quantum account(s)
token = IBMQ.save_account('9d9d67f30e979c94f79826bb53a391c1b0a822660c6f49755656e8afde78092d159903b13b96ae0bd97b92dd3b8dc4f4c58168280ac0eb516673b46d781ec8a9',
                         overwrite=True)

# Filter warnings
import warnings
warnings.filterwarnings('ignore')

# Database Utilities

In [74]:
def create_db_connection():
    con = sqlite3.connect('ibm_real_device_run_results.db')
    return con

def drop_table(con):
    cur = con.cursor()
    sql = '''
            DROP TABLE run_data
          '''
    cur.execute(sql)
    con.commit()

def create_table(con):
    cur = con.cursor()
    
    # counts_experimental = counts with error on real device
    # counts_ideal = counts without error on real device
    # counts_noisy_simulation = counts with error on simulator
    
    sql = '''
            CREATE TABLE IF NOT EXISTS run_data (id INTEGER PRIMARY KEY AUTOINCREMENT, backend TEXT, state_name TEXT, 
            tvd REAL, jsd REAL, hellinger REAL, counts_experimental TEXT, counts_ideal TEXT, counts_simulation TEXT)
          '''
    cur.execute(sql)
    con.commit()
    
def insert_into_table(con, backend, state_name, tvd, jsd, hellinger, counts_experimental, counts_ideal, counts_simulation):
    cur = con.cursor()
    sql = '''
            INSERT INTO run_data (backend, state_name, tvd, jsd, 
            hellinger, counts_experimental, counts_ideal, counts_simulation) 
            VALUES ("{}", "{}", {}, {}, {}, "{}", "{}", "{}")
          '''.format(backend, state_name, tvd, jsd, hellinger, counts_experimental, counts_ideal, counts_simulation)
    cur.execute(sql)
    con.commit()
    
def show_table_data(con):
    cur = con.cursor()
    sql = '''
            SELECT * FROM run_data
          '''
    res = cur.execute(sql)
    print(res.fetchall())
    
def check_data_for_some_backend_and_state_exists(con, backend, state_name):
    cur = con.cursor()
    cur.execute("SELECT id FROM run_data WHERE backend = ? AND state_name = ?", (backend, state_name))
    data = cur.fetchall()
    
    if len(data) == 0:
        return False
    return True

def update_table(counts_ideal, state_name):
    con = create_db_connection()
    cur = con.cursor()
    sql = '''UPDATE run_data SET counts_ideal = "{}" WHERE state_name = "{}"'''.format(counts_ideal, state_name)
    cur.execute(sql)
    con.commit()
    con.close()

In [75]:
con = create_db_connection()
create_table(con)

# Noise Simulation, Ideal State and Real Run

In [76]:
def add_missing_states(qubits, counts_experimental, counts_simulation, counts_ideal):
    # add missing binary states in both distributions
    # example, dist1 = {'0': 400} and dist2 = {'1': 600}
    # the follwoing for loop will make them both same size (i.e., with same keys)
    # new dist1 = {'0': 400, '1': 0} and dist2 = {'0': 0, '1': 600}
    for number in range(2**qubits):
        # following line converts an integer to a binary string 
        # the binary string length is fixed and it is number of qubits
        # example, if number of qubit is 4, integer 3 will be `0011`
        binary_state = '{0:b}'.format(number).zfill(qubits)

        if binary_state not in counts_experimental:
            counts_experimental[binary_state] = 0

        if binary_state not in counts_simulation:
            counts_simulation[binary_state] = 0
            
        if binary_state not in counts_ideal:
            counts_ideal[binary_state] = 0
            
    return counts_experimental, counts_simulation, counts_ideal

In [77]:
def calculate_tvd(counts_experimental, counts_simulation, N1, N2):
    # following loop actually computes the TVD between two distributions
    tvd = 0
    for key in counts_experimental:
        print(counts_experimental[key], counts_simulation[key])
        tvd = tvd + 0.5 * abs(counts_experimental[key]/N1 - counts_simulation[key]/N2)
    return tvd

def calculate_jsd(counts_experimental, counts_simulation):
    a = counts_experimental.values()
    b = counts_simulation.values()
    data_a = list(a)
    data_b = list(b)
    arr_a = np.array(data_a)
    arr_b = np.array(data_b)
    jsd = distance.jensenshannon(arr_a, arr_b)
    
    return jsd 

In [78]:
def all_error(circ, qubits, circ_name, con):  
    reset_error = 0.05 # this acts on single qubit gate, probability of flip a singe qubit P(reset)
    measure_error = 0.2 # P(measure error)
    gate_error = 0.05 # P(two qubit error)

    # calling error functions using from IBM library
    re_err = pauli_error([('X', reset_error),('I',1-reset_error)])
    meas_err = pauli_error([('X', measure_error),('I', 1-measure_error)])
    gate_err1 = pauli_error([('X', gate_error),('I', 1-gate_error)])
    gate_err2 = gate_err1.tensor(gate_err1)

    # errors on a noisy model
    all_err = NoiseModel()
    all_err.add_all_qubit_quantum_error(re_err,"reset")
    all_err.add_all_qubit_quantum_error(meas_err,"measure")
    all_err.add_all_qubit_quantum_error(gate_err1,["u1", "u2", "u3"])
    all_err.add_all_qubit_quantum_error(gate_err2, "cx")
    
    # T1 and T2 values for qubits 0-3
    T1s = np.random.normal(50e3, 10e3, qubits) # Sampled from normal distribution mean 50 microsec
    T2s = np.random.normal(70e3, 10e3, qubits)  # Sampled from normal distribution mean 50 microsec

    # Truncate random T2s <= T1s
    T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(qubits)])

    # Instruction times (in nanoseconds)
    time_u1 = 0   # virtual gate
    time_u2 = 50  # (single X90 pulse)
    time_u3 = 100 # (two X90 pulses)
    time_cx = 300
    time_reset = 1000  # 1 microsecond
    time_measure = 1000 # 1 microsecond

    # QuantumError objects
    errors_reset = [thermal_relaxation_error(t1, t2, time_reset)
                for t1, t2 in zip(T1s, T2s)]
    errors_measure = [thermal_relaxation_error(t1, t2, time_measure)
                  for t1, t2 in zip(T1s, T2s)]
    errors_u1  = [thermal_relaxation_error(t1, t2, time_u1)
              for t1, t2 in zip(T1s, T2s)]
    errors_u2  = [thermal_relaxation_error(t1, t2, time_u2)
              for t1, t2 in zip(T1s, T2s)]
    errors_u3  = [thermal_relaxation_error(t1, t2, time_u3)
              for t1, t2 in zip(T1s, T2s)]
    errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
             thermal_relaxation_error(t1b, t2b, time_cx))
              for t1a, t2a in zip(T1s, T2s)]
               for t1b, t2b in zip(T1s, T2s)]

    # Add errors to noise model
    noise_thermal = NoiseModel()
    for j in range(qubits):
        all_err.add_quantum_error(errors_reset[j], "reset", [j])
        all_err.add_quantum_error(errors_measure[j], "measure", [j])
        all_err.add_quantum_error(errors_u1[j], "u1", [j])
        all_err.add_quantum_error(errors_u2[j], "u2", [j])
        all_err.add_quantum_error(errors_u3[j], "u3", [j])
        for k in range(3):
            all_err.add_quantum_error(errors_cx[j][k], "cx", [j, k])
        
    p_gate = 0.1
    error_meas = pauli_error([('X',reset_error), ('I', 1 - reset_error)])

    noise_depolar = NoiseModel()
    all_err.add_all_qubit_quantum_error(error_meas, "measure", qubits)
    
    provider = IBMQ.load_account()
    get_provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
    backends = provider.backends()
    backend_names = ['ibmq_lima', 'ibmq_belem', 'ibmq_quito', 'ibmq_manila', 'ibmq_nairobi', 'ibmq_oslo']
    
    for name in backend_names:
        if check_data_for_some_backend_and_state_exists(con, name, circ_name) == True:
            continue
        
        backend = provider.get_backend(name)
        if(backend.configuration().n_qubits > 2):
            print(backend, end='\n')
            
            success_real, success_ideal, success_simulation = True, True, True
            
            noise_model = NoiseModel.from_backend(backend)
            coupling_map = backend.configuration().coupling_map
            basis_gates = noise_model.basis_gates
            noise_model = NoiseModel.from_backend(backend).to_dict()
            
            # real run with error 
            try:
                real_run = execute(
                    circ, backend= provider.get_backend(name), coupling_map=coupling_map, 
                    basis_gates=basis_gates, noise_model=noise_model
                ).result()
                counts_experimental = real_run.get_counts()
            except:
                print("Real run for backend = {} and circuit = {} failed!", name, circ_name)
                success_real = False
            
            # ideal run without error
            try:
                ideal_sim = AerSimulator()
                ideal_result = ideal_sim.run(circ).result()
                counts_ideal = ideal_result.get_counts()
            except:
                print("Ideal run for backend = {} and circuit = {} failed!", name, circ_name)
                success_ideal = False
            
            # simulation with error 
            try:
                sim_noise = AerSimulator(noise_model=all_err)
                tnoise = transpile(circ, sim_noise)
                result_all_err = sim_noise.run(tnoise).result()
                counts_simulation = result_all_err.get_counts()
            except:
                print("Simulation run for backend = {} and circuit = {} failed!", name, circ_name)
                success_simulation = False
    
            num_binary_states = 2**qubits

            # get the total counts for two dictionaries
            N1 = sum(counts_experimental.values())
            N2 = sum(counts_simulation.values())
            N3 = sum(counts_ideal.values())

            counts_experimental, counts_simulation, counts_ideal = add_missing_states(qubits, counts_experimental, counts_simulation, counts_ideal)

            tvd = calculate_tvd(counts_experimental, counts_simulation, N1, N2)
            jsd = calculate_jsd(counts_experimental, counts_simulation)
            hell = hellinger_distance(counts_experimental, counts_simulation)
            
            if success_ideal and success_real and success_simulation:
                insert_into_table(con, name, circ_name, tvd, jsd, hell, counts_experimental, counts_ideal, counts_simulation)

# Circuits

### Phase Code

In [79]:
#phase code
phase_q = QuantumRegister(5)
phase_c = ClassicalRegister(5)
phase_circ = QuantumCircuit(phase_q, phase_c)
phase_circ.h(phase_q[0])
phase_circ.initialize([1,0], 1)
phase_circ.x(phase_q[2])
phase_circ.initialize([1,0], 3)
phase_circ.h(phase_q[4])
phase_circ.h(phase_q[0:5])
phase_circ.cz(phase_q[0], phase_q[1])
phase_circ.h(phase_q[2])
phase_circ.h(phase_q[0])
phase_circ.cz(phase_q[1], phase_q[2])
phase_circ.h(phase_q[1])
phase_circ.cz(phase_q[2], phase_q[3])
phase_circ.h(phase_q[2])
phase_circ.cz(phase_q[3], phase_q[4])
phase_circ.h(phase_q[3])
phase_circ.h(phase_q[4])
phase_circ.barrier()
phase_circ.measure(phase_q[0:4], phase_c[0:4])
display(phase_circ.draw())

┌───┐       ┌───┐     ┌───┐                ░ ┌─┐         
q14_0: ───────┤ H ├───────┤ H ├──■──┤ H ├────────────────░─┤M├─────────
       ┌──────┴───┴──────┐├───┤  │  └───┘┌───┐           ░ └╥┘┌─┐      
q14_1: ┤ Initialize(1,0) ├┤ H ├──■────■──┤ H ├───────────░──╫─┤M├──────
       └──────┬───┬──────┘├───┤┌───┐  │  └───┘┌───┐      ░  ║ └╥┘┌─┐   
q14_2: ───────┤ X ├───────┤ H ├┤ H ├──■────■──┤ H ├──────░──╫──╫─┤M├───
       ┌──────┴───┴──────┐├───┤└───┘       │  └───┘┌───┐ ░  ║  ║ └╥┘┌─┐
q14_3: ┤ Initialize(1,0) ├┤ H ├────────────■────■──┤ H ├─░──╫──╫──╫─┤M├
       └──────┬───┬──────┘├───┤                 │  ├───┤ ░  ║  ║  ║ └╥┘
q14_4: ───────┤ H ├───────┤ H ├─────────────────■──┤ H ├─░──╫──╫──╫──╫─
              └───┘       └───┘                    └───┘ ░  ║  ║  ║  ║ 
c14: 5/═════════════════════════════════════════════════════╩══╩══╩══╩═
                                                            0  1  2  3

### Bit Code

In [80]:
#bit code
bit_q  = QuantumRegister(5)
bit_c = ClassicalRegister(5)
bit_circ = QuantumCircuit(bit_q,bit_c)
bit_circ.barrier()
bit_circ.initialize([1,0], 1)
bit_circ.x(bit_q[2])
bit_circ.initialize([1,0], 3)
bit_circ.barrier()

bit_circ.cx(bit_q[0], bit_q[1])
bit_circ.cx(bit_q[2], bit_q[1])
bit_circ.cx(bit_q[2], bit_q[3])
bit_circ.cx(bit_q[4], bit_q[3])
bit_circ.barrier()
bit_circ.measure(bit_q[0:5],bit_c[0:5])
display(bit_circ.draw())

░                     ░                      ░ ┌─┐            
q15_0: ─░─────────────────────░───■──────────────────░─┤M├────────────
        ░ ┌─────────────────┐ ░ ┌─┴─┐┌───┐           ░ └╥┘┌─┐         
q15_1: ─░─┤ Initialize(1,0) ├─░─┤ X ├┤ X ├───────────░──╫─┤M├─────────
        ░ └──────┬───┬──────┘ ░ └───┘└─┬─┘           ░  ║ └╥┘┌─┐      
q15_2: ─░────────┤ X ├────────░────────■────■────────░──╫──╫─┤M├──────
        ░ ┌──────┴───┴──────┐ ░           ┌─┴─┐┌───┐ ░  ║  ║ └╥┘┌─┐   
q15_3: ─░─┤ Initialize(1,0) ├─░───────────┤ X ├┤ X ├─░──╫──╫──╫─┤M├───
        ░ └─────────────────┘ ░           └───┘└─┬─┘ ░  ║  ║  ║ └╥┘┌─┐
q15_4: ─░─────────────────────░──────────────────■───░──╫──╫──╫──╫─┤M├
        ░                     ░                      ░  ║  ║  ║  ║ └╥┘
c15: 5/═════════════════════════════════════════════════╩══╩══╩══╩══╩═
                                                        0  1  2  3  4

### Swap QAOA 

In [81]:
swap_qaoa_q = QuantumRegister(4)
swap_qaoa_c = ClassicalRegister(4)
swap_qaoa_circ = QuantumCircuit(swap_qaoa_q, swap_qaoa_c)
swap_qaoa_circ.h(swap_qaoa_q[0:4])
swap_qaoa_circ.cx(swap_qaoa_q[0], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[3])
swap_qaoa_circ.rz(np.pi/2, 1)
swap_qaoa_circ.rz(np.pi/2, 2)
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[0])
swap_qaoa_circ.cx(swap_qaoa_q[3], swap_qaoa_q[2])
swap_qaoa_circ.cx(swap_qaoa_q[0], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[3])
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[2])
swap_qaoa_circ.rz(np.pi/2, 3)
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[2])
swap_qaoa_circ.cx(swap_qaoa_q[0], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[3])
swap_qaoa_circ.rz(np.pi/2, 1)
swap_qaoa_circ.rz(np.pi/2, 3)
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[0])
swap_qaoa_circ.cx(swap_qaoa_q[3], swap_qaoa_q[2])
swap_qaoa_circ.cx(swap_qaoa_q[0], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[3])
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[2])
swap_qaoa_circ.rz(np.pi/2, 2)
swap_qaoa_circ.cx(swap_qaoa_q[2], swap_qaoa_q[1])
swap_qaoa_circ.cx(swap_qaoa_q[1], swap_qaoa_q[2])
swap_qaoa_circ.rz(np.pi/3, 0)
swap_qaoa_circ.rz(np.pi/3, 1)
swap_qaoa_circ.rz(np.pi/3, 2)
swap_qaoa_circ.rz(np.pi/3, 3)
swap_qaoa_circ.barrier()
swap_qaoa_circ.measure(swap_qaoa_q[0:4], swap_qaoa_c[0:4])
display(swap_qaoa_circ.draw())

┌───┐                ┌───┐                                          »
q16_0: ┤ H ├──■─────────────┤ X ├──■─────────────────────────■─────────────»
       ├───┤┌─┴─┐┌─────────┐└─┬─┘┌─┴─┐           ┌───┐     ┌─┴─┐┌─────────┐»
q16_1: ┤ H ├┤ X ├┤ Rz(π/2) ├──■──┤ X ├─────■─────┤ X ├──■──┤ X ├┤ Rz(π/2) ├»
       ├───┤└───┘├─────────┤┌───┐└───┘   ┌─┴─┐   └─┬─┘┌─┴─┐└───┘└─────────┘»
q16_2: ┤ H ├──■──┤ Rz(π/2) ├┤ X ├──■─────┤ X ├─────■──┤ X ├──■─────────────»
       ├───┤┌─┴─┐└─────────┘└─┬─┘┌─┴─┐┌──┴───┴──┐     └───┘┌─┴─┐┌─────────┐»
q16_3: ┤ H ├┤ X ├─────────────■──┤ X ├┤ Rz(π/2) ├──────────┤ X ├┤ Rz(π/2) ├»
       └───┘└───┘                └───┘└─────────┘          └───┘└─────────┘»
c16: 4/════════════════════════════════════════════════════════════════════»
                                                                           »
«       ┌───┐     ┌─────────┐                                 ░ ┌─┐         
«q16_0: ┤ X ├──■──┤ Rz(π/3) ├─────────────────────────────────░─┤M├─────────
«       └─┬─┘┌─┴─┐└─────────┘           ┌───┐     ┌─────────┐ ░ └╥┘┌─┐      
«q16_1: ──■──┤ X ├─────■────────────────┤ X ├──■──┤ Rz(π/3) ├─░──╫─┤M├──────
«       ┌───┐└───┘   ┌─┴─┐   ┌─────────┐└─┬─┘┌─┴─┐├─────────┤ ░  ║ └╥┘┌─┐   
«q16_2: ┤ X ├──■─────┤ X ├───┤ Rz(π/2) ├──■──┤ X ├┤ Rz(π/3) ├─░──╫──╫─┤M├───
«       └─┬─┘┌─┴─┐┌──┴───┴──┐└─────────┘     └───┘└─────────┘ ░  ║  ║ └╥┘┌─┐
«q16_3: ──■──┤ X ├┤ Rz(π/3) ├─────────────────────────────────░──╫──╫──╫─┤M├
«            └───┘└─────────┘                                 ░  ║  ║  ║ └╥┘
«c16: 4/═════════════════════════════════════════════════════════╩══╩══╩══╩═
«                                                                0  1  2  3

### Vanilla QAOA 

In [82]:
vanilla_qaoa_q = QuantumRegister(3)
vanilla_qaoa_c = ClassicalRegister(3)
vanilla_qaoa_circ = QuantumCircuit(vanilla_qaoa_q, vanilla_qaoa_c)
vanilla_qaoa_circ.h(vanilla_qaoa_q[0:3])
vanilla_qaoa_circ.cx(vanilla_qaoa_q[0], vanilla_qaoa_q[1])
vanilla_qaoa_circ.rz(np.pi/2, 1)
vanilla_qaoa_circ.cx(vanilla_qaoa_q[0], vanilla_qaoa_q[1])
vanilla_qaoa_circ.cx(vanilla_qaoa_q[0], vanilla_qaoa_q[2])
vanilla_qaoa_circ.rz(np.pi/2, 2)
vanilla_qaoa_circ.cx(vanilla_qaoa_q[0], vanilla_qaoa_q[2])
vanilla_qaoa_circ.cx(vanilla_qaoa_q[1], vanilla_qaoa_q[2])
vanilla_qaoa_circ.rz(np.pi/2, 2)
vanilla_qaoa_circ.cx(vanilla_qaoa_q[1], vanilla_qaoa_q[2])
vanilla_qaoa_circ.rz(np.pi/3, 0)
vanilla_qaoa_circ.rz(np.pi/3, 1)
vanilla_qaoa_circ.rz(np.pi/3, 2)
vanilla_qaoa_circ.barrier()
vanilla_qaoa_circ.measure(vanilla_qaoa_q[0:3], vanilla_qaoa_c[0:3])
display(vanilla_qaoa_circ.draw())

┌───┐                                          ┌─────────┐           »
q17_0: ┤ H ├──■───────────────■────■───────────────■──┤ Rz(π/3) ├───────────»
       ├───┤┌─┴─┐┌─────────┐┌─┴─┐  │               │  └─────────┘           »
q17_1: ┤ H ├┤ X ├┤ Rz(π/2) ├┤ X ├──┼───────────────┼───────■────────────────»
       ├───┤└───┘└─────────┘└───┘┌─┴─┐┌─────────┐┌─┴─┐   ┌─┴─┐   ┌─────────┐»
q17_2: ┤ H ├─────────────────────┤ X ├┤ Rz(π/2) ├┤ X ├───┤ X ├───┤ Rz(π/2) ├»
       └───┘                     └───┘└─────────┘└───┘   └───┘   └─────────┘»
c17: 3/═════════════════════════════════════════════════════════════════════»
                                                                            »
«                        ░ ┌─┐      
«q17_0: ─────────────────░─┤M├──────
«            ┌─────────┐ ░ └╥┘┌─┐   
«q17_1: ──■──┤ Rz(π/3) ├─░──╫─┤M├───
«       ┌─┴─┐├─────────┤ ░  ║ └╥┘┌─┐
«q17_2: ┤ X ├┤ Rz(π/3) ├─░──╫──╫─┤M├
«       └───┘└─────────┘ ░  ║  ║ └╥┘
«c17: 3/════════════════════╩══╩══╩═
«                           0  1  2

### Hamiltonian Circuit

In [83]:
hamilton_q = QuantumRegister(3)
hamilton_c = ClassicalRegister(3)
hamilton_circ = QuantumCircuit(hamilton_q, hamilton_c)
hamilton_circ.h(hamilton_q[0:4])
hamilton_circ.rz(np.pi/2, hamilton_q[0:4])
hamilton_circ.h(hamilton_q[0:4])
hamilton_circ.cx(hamilton_q[0], hamilton_q[1])
hamilton_circ.rz(np.pi/3, 1)
hamilton_circ.cx(hamilton_q[0], hamilton_q[1])
hamilton_circ.cx(hamilton_q[1], hamilton_q[2])
hamilton_circ.rz(np.pi/3, 2)
hamilton_circ.cx(hamilton_q[1], hamilton_q[2])
hamilton_circ.barrier()
hamilton_circ.measure(hamilton_q[0:3], hamilton_c[0:3])
display(hamilton_circ.draw())

┌───┐┌─────────┐┌───┐                                           ░ ┌─┐   »
q18_0: ┤ H ├┤ Rz(π/2) ├┤ H ├──■───────────────■────────────────────────░─┤M├───»
       ├───┤├─────────┤├───┤┌─┴─┐┌─────────┐┌─┴─┐                      ░ └╥┘┌─┐»
q18_1: ┤ H ├┤ Rz(π/2) ├┤ H ├┤ X ├┤ Rz(π/3) ├┤ X ├──■───────────────■───░──╫─┤M├»
       ├───┤├─────────┤├───┤└───┘└─────────┘└───┘┌─┴─┐┌─────────┐┌─┴─┐ ░  ║ └╥┘»
q18_2: ┤ H ├┤ Rz(π/2) ├┤ H ├─────────────────────┤ X ├┤ Rz(π/3) ├┤ X ├─░──╫──╫─»
       └───┘└─────────┘└───┘                     └───┘└─────────┘└───┘ ░  ║  ║ »
c18: 3/═══════════════════════════════════════════════════════════════════╩══╩═»
                                                                          0  1 »
«          
«q18_0: ───
«          
«q18_1: ───
«       ┌─┐
«q18_2: ┤M├
«       └╥┘
«c18: 3/═╩═
«        2

### GHZ Circuit

In [84]:
q_ghz = QuantumRegister(3)
c_ghz = ClassicalRegister(3)
ghz_circ = QuantumCircuit(q_ghz, c_ghz)
ghz_circ.h(q_ghz[0])
ghz_circ.cx(q_ghz[0], q_ghz[1])
ghz_circ.cx(q_ghz[1], q_ghz[2])
ghz_circ.measure(q_ghz[0:3], c_ghz[0:3])
ghz_circ.draw()

┌───┐          ┌─┐      
q19_0: ┤ H ├──■───────┤M├──────
       └───┘┌─┴─┐     └╥┘┌─┐   
q19_1: ─────┤ X ├──■───╫─┤M├───
            └───┘┌─┴─┐ ║ └╥┘┌─┐
q19_2: ──────────┤ X ├─╫──╫─┤M├
                 └───┘ ║  ║ └╥┘
c19: 3/════════════════╩══╩══╩═
                       0  1  2

### Mermin Bell Circuit

In [85]:
mer_bell = QuantumRegister(3)
mer_bell_c =  ClassicalRegister(3)
mer_bell_circ = QuantumCircuit(mer_bell, mer_bell_c)
mer_bell_circ.rx(-pi/2, mer_bell[0])
mer_bell_circ.cx(mer_bell[0], mer_bell[1])
mer_bell_circ.cx(mer_bell[1], mer_bell[2])
mer_bell_circ.h(mer_bell[1])
mer_bell_circ.h(mer_bell[2])
mer_bell_circ.cx(mer_bell[0], mer_bell[2])
mer_bell_circ.cx(mer_bell[1], mer_bell[2])
mer_bell_circ.cx(mer_bell[2], mer_bell[0])
mer_bell_circ.cx(mer_bell[1], mer_bell[0])
mer_bell_circ.s(mer_bell[2])
mer_bell_circ.s(mer_bell[0])
mer_bell_circ.h(mer_bell[2])
mer_bell_circ.cz(mer_bell[0], mer_bell[1])
mer_bell_circ.h(mer_bell[0])
mer_bell_circ.s(mer_bell[1])
mer_bell_circ.h(mer_bell[1])
mer_bell_circ.measure(mer_bell[0:3], mer_bell_c[0:3])
display(mer_bell_circ.draw())

┌──────────┐                         ┌───┐┌───┐┌───┐   ┌───┐     ┌─┐   
q20_0: ┤ Rx(-π/2) ├──■──────────────■───────┤ X ├┤ X ├┤ S ├─■─┤ H ├─────┤M├───
       └──────────┘┌─┴─┐     ┌───┐  │       └─┬─┘└─┬─┘└───┘ │ ├───┤┌───┐└╥┘┌─┐
q20_1: ────────────┤ X ├──■──┤ H ├──┼────■────┼────■────────■─┤ S ├┤ H ├─╫─┤M├
                   └───┘┌─┴─┐├───┤┌─┴─┐┌─┴─┐  │  ┌───┐┌───┐┌─┐└───┘└───┘ ║ └╥┘
q20_2: ─────────────────┤ X ├┤ H ├┤ X ├┤ X ├──■──┤ S ├┤ H ├┤M├───────────╫──╫─
                        └───┘└───┘└───┘└───┘     └───┘└───┘└╥┘           ║  ║ 
c20: 3/═════════════════════════════════════════════════════╩════════════╩══╩═
                                                            2            0  1

# Calculate all errors

In [86]:
all_error(phase_circ, len(phase_q), 'phase_circ', con)

ibmqfactory.load_account:WARNING:2022-08-13 11:02:08,094: Credentials are already in use. The existing account in the session will be replaced.


In [87]:
all_error(bit_circ, len(bit_q), 'bit_circ', con)

ibmqfactory.load_account:WARNING:2022-08-13 11:02:14,010: Credentials are already in use. The existing account in the session will be replaced.


In [88]:
all_error(swap_qaoa_circ, len(swap_qaoa_q), 'swap_qaoa_circ', con)

ibmqfactory.load_account:WARNING:2022-08-13 11:02:18,720: Credentials are already in use. The existing account in the session will be replaced.


In [89]:
all_error(vanilla_qaoa_circ, len(vanilla_qaoa_q), 'vanilla_qaoa_circ', con)

ibmqfactory.load_account:WARNING:2022-08-13 11:02:22,919: Credentials are already in use. The existing account in the session will be replaced.


In [90]:
all_error(hamilton_circ, len(hamilton_q), 'hamilton_circ', con)

ibmqfactory.load_account:WARNING:2022-08-13 11:02:26,810: Credentials are already in use. The existing account in the session will be replaced.


In [91]:
all_error(ghz_circ, len(q_ghz), 'ghz_circ', con)

ibmqfactory.load_account:WARNING:2022-08-13 11:02:30,495: Credentials are already in use. The existing account in the session will be replaced.


In [92]:
all_error(mer_bell_circ, len(mer_bell), 'mermin_circ', con)

ibmqfactory.load_account:WARNING:2022-08-13 11:02:33,701: Credentials are already in use. The existing account in the session will be replaced.


In [93]:
con.close()